# Mining frequent itemsets and association rules with `mlxtend`

En este notebook utilizaremos mlxtend (http://rasbt.github.io/mlxtend/) y pandas (https://pandas.pydata.org/9) para buscar frequent itemsets y reglas de asociación. Comenzamos importando los paquetes necesarios:

In [ ]:
import pandas as pd

## Apriori

En primer lugar, utilizaremos el algoritmo Apriori. La función `apriori` (http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/) de esta librería espera los datos de las transacciones en un dataframe pandas con una "one-hot encoding". Supongamos que tenemos la siguiente lista de transacciones (lista de listas):

In [ ]:
dataset = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]

Podemos transformar esta lista de listas a un dataframe pandas con la codificación adecuada utilizando el `TransactionEncoder`:

In [ ]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

Antes de comenzar el análisis, vamos a obtener una gráfica con los 5 productos más frecuentes:

In [ ]:
df.sum(axis=0).sort_values(ascending=False)[0:5].plot(kind="bar")

Vamos a buscar los itemsets con un soporte mínimo del 60%:

In [ ]:
from mlxtend.frequent_patterns import apriori

apriori(df, min_support=0.6)

Por defecto, la función devuelve los índices de las columnas de los ítems, lo cual puede ser útil para análisis posteriores automáticos. Sin embargo, para mejora rla legibilidad, podemos utilizar `use_colnames=True` y obtener los nombres:

In [ ]:
apriori(df, min_support=0.6, use_colnames=True)

El resultado de la librería es un DataFrame de pandas, que podemos utilizar para filtrar los resultados fácilmente. Por ejemplo, vamos a quedarnos solo con los itemsets de longitud 2 con un soporte mínimo del 80%. Primero, creamos los frequent itemsets con `apriori` y luego le añadimos una columna al DataFrame con la longitud de cada itemset:

In [ ]:
frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

Y ahora podemos filtrar el resultado fácilmente:

In [ ]:
frequent_itemsets[ (frequent_itemsets['length'] == 2) &
                   (frequent_itemsets['support'] >= 0.8) ]

# Reglas de asociación

Ahora que tenemos los frequent itemsets, podemos descubrir reglas de asociación a partir de ellos. La función `generate_rules` de la librería (http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/) toma como entrada el DataFrame de frequent itemsets producido por la función `apriori`. La función `generate_rules` permite espefificar la métrica que se quiere emplear (confidence, lift, etc.) y el umbral correspondiente. Vamos a obtener las reglas cuyo nivel de confianza esté por encima del 70%:

In [ ]:
from mlxtend.frequent_patterns import association_rules

frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))

association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

Y ahora, vamos a quedarnos con las reglas con un lift de al menos 1.2:

In [ ]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)
rules

Vamos a utilizar los filtros de los DataFrames de pandas para refinar los resultados. Supongamos que queremos reglas con: 
 - Por lo menos 2 antecedentes.
 - Una confianza superior al 75%.
 - Un lift > 1.2.

En primer lugar, añadimos al DataFrame `rules` la longitud de los antecedentes:

In [ ]:
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules

In [ ]:
rules[ (rules['antecedent_len'] >= 2) &
       (rules['confidence'] > 0.75) &
       (rules['lift'] > 1.2) ]

También podemos filtrar las reglas basándonos en las columnas "antecedents" o "consequents":

In [ ]:
rules[rules['antecedents'] == {'Eggs', 'Kidney Beans'}]

# Ejercicios propuestos

## 1. Probar otros algoritmos de búsqueda de frequent itemsets

La librería mlxtend tiene otros dos algoritmos de búsqueda de frequent itemsets (http://rasbt.github.io/mlxtend/api_subpackages/mlxtend.frequent_patterns/): FP-Growth (http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/fpgrowth/) y FP-Max (http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/fpmax/). Prueba ambos algoritmos con el pequeño dataset de prueba, incluyendo la fase de genereación de reglas.

## 2. Aplicar los algoritmos a datasets reales (*entrega en Moovi*)

En la carpeta `frequent-itemsets` hay dos ficheros con datos reales: `groceries.csv` y `store_data.csv`. Utiliza estos ficheros (uno de ellos o ambos) para desarrollar un caso de estudio con un pequeño análisis de los resultados. Haz esto en un notebook Jupyter separado.

A continuación encontrarás un ejemplo de cómo cargar estos ficheros a una lista de listas que podrás utilizar para analizar con mlxtend:

In [ ]:
fh = open('groceries.csv')
groceries = []
for line in fh:
    groceries.append(line.replace('\n', '').split(','))
fh.close()

te = TransactionEncoder()
te_ary = te.fit(groceries).transform(groceries)
df = pd.DataFrame(te_ary, columns=te.columns_)
df